## Accuracy

In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  
os.environ["CUDA_VISIBLE_DEVICES"]="2" 

In [2]:
from catalyst.dl.callbacks import Callback, AccuracyCallback
from catalyst.dl.experiments import SupervisedRunner

from collections import OrderedDict, defaultdict

import torch
import numpy as np

In [3]:
loaders = OrderedDict()

In [4]:
import sys
sys.path.append('../')
import dataloaders

import torchvision.datasets as datasets
from torchvision import transforms

# DATA_ROOT = "/workspace/raid/data/datasets"
DATA_ROOT = "/gpfs/gpfs0/e.ponomarev/"
dataset_name = 'imagenet'

bs = 128
num_workers = 32

if dataset_name == 'cifar10':
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
    DATA_PATH = "{}/cifar10".format(DATA_ROOT)

    loaders["valid"] = torch.utils.data.DataLoader(
        datasets.CIFAR10(root=DATA_PATH, train=False, transform=transforms.Compose([
            transforms.ToTensor(),
            normalize,
        ]),  download = True),
        batch_size=bs, shuffle=False,
        num_workers=num_workers, pin_memory=True)
elif dataset_name == 'imagenet':
    loaders["valid"] = dataloaders.get_loader(batch_size=bs,
                                        data_name = 'imagenet',
                                        data_root = DATA_ROOT,
                                        num_workers = num_workers, 
                                        pin_memory = True)['val']
    loaders["train"] = dataloaders.get_loader(batch_size=bs,
                                    data_name = 'imagenet',
                                    data_root = DATA_ROOT,
                                    num_workers = num_workers, 
                                    pin_memory = True)['train']

Building imagenet data loader with 32 workers
Building imagenet data loader with 32 workers


In [5]:
class ValCallback(Callback):
    
    
    def __init__(self, out_dir=None, out_prefix=None):
        self.out_dir = out_dir
        self.out_prefix = out_prefix
        self.predictions = defaultdict(lambda: [])
        self._keys_from_state = ["out_dir", "out_prefix"]

    def on_stage_start(self, state):
        for key in self._keys_from_state:
            value = getattr(state, key, None)
            if value is not None:
                setattr(self, key, value)
        # assert self.out_prefix is not None
        if self.out_dir is not None:
            self.out_prefix = str(self.out_dir) + "/" + str(self.out_prefix)
        if self.out_prefix is not None:
            os.makedirs(os.path.dirname(self.out_prefix), exist_ok=True)

    def on_loader_start(self, state):
        self.predictions = defaultdict(lambda: [])
        self.inputs = defaultdict(lambda: [])

    def on_batch_end(self, state):
        dct = state.output
        dct = {key: value.detach().cpu().numpy() for key, value in dct.items()}
        for key, value in dct.items():
            self.predictions[key].append(value)
            
        dct = state.input
        dct = {key: value.detach().cpu().numpy() for key, value in dct.items()}
        for key, value in dct.items():
            self.inputs[key].append(value)

    def on_loader_end(self, state):
        from catalyst.dl.metrics import accuracy
        self.predictions = {
            key: np.concatenate(value, axis=0)
            for key, value in self.predictions.items()
        }
        self.inputs = {
            key: np.concatenate(value, axis=0)
            for key, value in self.inputs.items()
        }
        y_true = torch.tensor(self.inputs['targets'])
        y_pred = torch.tensor(self.predictions['logits'])
        self.accuracy_score = torch.stack(accuracy(y_pred,y_true,topk=(1,5))).numpy().astype('float32')
        self.accuracy_score = np.squeeze(self.accuracy_score)

In [ ]:
import sys
sys.path.append('../')

from model_utils import load_model
from torchvision.models import resnet34

# MODEL_NAME = 'vgg16_imagenet'
# MODEL_NAME = 'resnet50_imagenet'
# MODEL_NAME = 'resnet18_imagenet'
MODEL_NAME = 'resnet34_imagenet'

# MODEL_NAME = 'faster_rcnn_vgg16
# MODEL_NAME = 'faster_rcnn_resnet50'


# # Uncomment if MODEL_NAME = 'faster_rcnn_resnet50'
# sys.path.append('/workspace/home/jgusak/maxvol_objects/facebook_frcnn/')
# import maskrcnn_benchmark

# model = load_model(MODEL_NAME)

model = resnet34(pretrained = True)

In [ ]:
lr = 0.002
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),weight_decay=1e-6,lr=lr)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer)

# model runner
runner = SupervisedRunner()

# model training
runner.train(
    model=model,
    criterion=criterion,
    callbacks=[
        AccuracyCallback(accuracy_args = [1,5]), # top (1,5) accuracy  on batch
#         ValCallback()
    ],
    optimizer=optimizer,
    scheduler=scheduler,
    loaders=loaders,
    logdir='./logs2',
    num_epochs=15,
    verbose=True,
    valid_loader='valid'
    
)



0/15 * Epoch (valid): 100% 391/391 [01:04<00:00,  6.10it/s, _timers/_fps=567.967, accuracy01=46.250, accuracy05=86.250, loss=1.938]   
0/15 * Epoch (train):  19% 1886/10010 [10:09<49:14,  2.75it/s, _timers/_fps=3634.997, accuracy01=31.250, accuracy05=52.344, loss=3.495]   

/trinity/shared/opt/python-3.7.1/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:754: UserWarning:

Possibly corrupt EXIF data.  Expecting to read 19660800 bytes but only got 0. Skipping tag 0

/trinity/shared/opt/python-3.7.1/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:754: UserWarning:

Possibly corrupt EXIF data.  Expecting to read 18481152 bytes but only got 0. Skipping tag 0

/trinity/shared/opt/python-3.7.1/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:754: UserWarning:

Possibly corrupt EXIF data.  Expecting to read 37093376 bytes but only got 0. Skipping tag 0

/trinity/shared/opt/python-3.7.1/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:754: UserWarning:

Possibly corrupt EXIF data.  Expecting to read 39976960 bytes but only got 0. Skipping tag 0

/trinity/shared/opt/python-3.7.1/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:754: UserWarning:

Possibly corrupt EXIF data.  Expecting to read 34865152 bytes but only got 0. Skipping tag 0

/trinity/shared

0/15 * Epoch (train):  62% 6240/10010 [32:17<11:21,  5.53it/s, _timers/_fps=2959.662, accuracy01=38.281, accuracy05=60.938, loss=2.829]   

/trinity/shared/opt/python-3.7.1/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:771: UserWarning:

Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 



0/15 * Epoch (train):  63% 6263/10010 [32:25<11:11,  5.58it/s, _timers/_fps=3340.453, accuracy01=42.188, accuracy05=69.531, loss=2.570]

...

7/15 * Epoch (train):  55% 5520/10010 [17:54<17:10,  4.36it/s, _timers/_fps=3254.098, accuracy01=60.156, accuracy05=78.906, loss=1.681]

/trinity/shared/opt/python-3.7.1/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:754: UserWarning:

Possibly corrupt EXIF data.  Expecting to read 2555904 bytes but only got 0. Skipping tag 0



7/15 * Epoch (train):  67% 6728/10010 [21:48<09:57,  5.50it/s, _timers/_fps=3191.196, accuracy01=60.938, accuracy05=78.906, loss=1.763]  

/trinity/shared/opt/python-3.7.1/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:771: UserWarning:

Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 



7/15 * Epoch (train):  70% 6995/10010 [22:40<08:50,  5.69it/s, _timers/_fps=3499.354, accuracy01=64.844, accuracy05=82.031, loss=1.490]

In [8]:
scores0 = run(model)

0/1 * Epoch (infer): 100% 391/391 [01:09<00:00,  6.08it/s, _timers/_fps=684.177, accuracy01=46.250, accuracy03=80.000, accuracy05=86.250]  


In [9]:
scores0

array([73.314   , 87.728004, 91.420006], dtype=float32)